In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd
import os
import seaborn

In [2]:
def gettrueutil(rank,n,trueutil):
    order=np.argsort(trueutil)[::-1]
    return np.sum(np.exp(trueutil[rank[:n]]))/sum(np.exp(trueutil[order[:n]]))

In [3]:
def getutil(b,n):
    return np.sum(np.exp(np.sort(b)[-n:]))

In [4]:
def createlabel(f,newname='util'):
    name=f.split()
    name[0]=newname
    return ' '.join(name)

In [5]:
def rank(array):
    temp = array.argsort()[::-1]
    ranks = np.empty(len(array), int)
    ranks[temp] = np.arange(len(array))+1
    return ranks

In [6]:
def generateutils(table,trueutil):
    data={}
    data['iteration']=[]
    for i in range(1,101):
            data['iteration']+=[i]*51
    data['respondent']=[x for x in range(20,1040,20)]*100
    vals=table.get_values()[1:,2:]
    n,m=np.shape(vals)
    data['util3']=[]
    data['util10']=[]
    for i in range(n):
        rank=np.argsort(vals[i])[::-1]
        data['util3'].append(gettrueutil(rank,3,trueutil))
        data['util10'].append(gettrueutil(rank,10,trueutil))
    newtable=pd.DataFrame(data)
    return newtable

In [7]:
def getfiles(start="bhist",items='120'):
    listoffiles=[]
    where='../DataDump'
    for file in os.listdir(where):
        if file.startswith(start) and items in file:
            listoffiles.append(file)
    return listoffiles,where

In [8]:
def utilall(items='120'):
    files,where=getfiles(items=items)
    trueutil=pd.read_csv('../MABMaxDiff/HB_'+items+'items.csv').get_values().mean(0)[2:]
    for file in files:
        table=pd.read_csv(where+'/'+file)
        newtable=generateutils(table,trueutil)
        newtable.to_csv(where+'/'+createlabel(file),index=False)

In [17]:
#utilall()
#utilall('40')
#utilall('300')

In [10]:
def generatevalrem(table1,table2):
    data={}
    data['iteration']=[]
    for i in range(1,101):
            data['iteration']+=[i]*51
    data['respondent']=[x for x in range(20,1040,20)]*100
    vals1=table1.get_values()[1:,2:]
    vals2=table2.get_values()[1:,2:]
    n,m=np.shape(vals1)
    data['valrem3']=[]
    data['valrem10']=[]
    data['util3']=[]
    data['util10']=[]
    for i in range(n):
        data['valrem3'].append(getvalrem(vals1[i],vals2[i],3))
        data['valrem10'].append(getvalrem(vals1[i],vals2[i],10))
        data['util3'].append(getutil(vals1[i],3))
        data['util10'].append(getutil(vals1[i],10))
    newtable=pd.DataFrame(data)
    return newtable

In [11]:
def getvalrem(b,se,n,iters=1000):
    rank=np.argsort(b)[::-1]
    vals=np.random.multivariate_normal(b,np.diag(se**2),iters)
    valstar=np.sum(np.exp(vals[:,rank[:n]]),1)
    valmax=np.sum(np.exp(np.sort(vals,1)[:,-n:]),1)
    vrhist=(valmax-valstar)/valstar
    return np.percentile(vrhist,95)

In [12]:
def valremall(items='120'):
    files1,where=getfiles(items=items)
    files2,where=getfiles('sehist',items=items)
    for file1,file2 in zip(files1,files2):
        table1=pd.read_csv(where+'/'+file1)
        table2=pd.read_csv(where+'/'+file2)
        newtable=generatevalrem(table1,table2)
        newtable.to_csv(where+'/'+createlabel(file1,'valrem'),index=False)

In [14]:
items='120'
files1,where=getfiles(items=items)
files2,where=getfiles('sehist',items=items)

In [19]:
file1=files1[10]
file2=files2[10]
table1=pd.read_csv(where+'/'+file1)
table2=pd.read_csv(where+'/'+file2)
vals1=table1.get_values()[1:,2:]
vals2=table2.get_values()[1:,2:]


In [65]:
i=4
b,se,n=vals1[i],vals2[i],3
rank=np.argsort(b)[::-1]
vals=np.random.multivariate_normal(b,np.diag(se**2),10000)
valstar=np.sum(np.exp(vals[:,rank[:n]]),1)
valmax=np.sum(np.exp(np.sort(vals,1)[:,-n:]),1)
vrhist=(valmax-valstar)/valstar
vrhist951=np.percentile(vrhist,95)
plt.subplot('121')
plt.hist(vrhist)
plt.axvline(vrhist951,0,1,color='k')
plt.xlim(0,.25)
plt.ylim(0,10000)
i=10
b,se,n=vals1[i],vals2[i],3
rank=np.argsort(b)[::-1]
vals=np.random.multivariate_normal(b,np.diag(se**2),10000)
valstar=np.sum(np.exp(vals[:,rank[:n]]),1)
valmax=np.sum(np.exp(np.sort(vals,1)[:,-n:]),1)
vrhist=(valmax-valstar)/valstar
vrhist95=np.percentile(vrhist,95)
plt.subplot('122')
plt.hist(vrhist)
plt.axvline(vrhist95,0,1,color='k')
plt.xlim(0,.25)
plt.ylim(0,10000)
plt.savefig('valremhist.pdf')
plt.show()

In [68]:
vrhist95


0.0075873118884629458

In [36]:
plt.axvline?

In [21]:
#valremall()
#valremall('40')
#valremall('300')

In [94]:
def stopingutils(items='120'):
    files1,where=getfiles('valrem',items=items)
    files2,where=getfiles('util',items=items)
    for file1,file2 in zip(files1,files2):
        table1=pd.read_csv(where+'/'+file1)
        table2=pd.read_csv(where+'/'+file2)
        newtable=generatestop(table1,table2)
        newtable.to_csv(where+'/'+createlabel(file1,'stop'),index=False)

In [99]:
def generatestop(valrem,util):
    data={}
    data['iteration']=[i for i in range(1,101)]
    data['stop5rem3']=[]
    data['stop2rem3']=[]
    data['stop5rem10']=[]
    data['stop2rem10']=[]
    data['util5rem3']=[]
    data['util2rem3']=[]
    data['util5rem10']=[]
    data['util2rem10']=[]
    for i in range(1,101):
        for value in [2,5]:
            for num in ['3','10']:
                loc,trueutil=getstoputil(valrem,util,i,'valrem'+num,'util'+num,value/100)
                data['stop'+str(value)+'rem'+num].append(loc)
                data['util'+str(value)+'rem'+num].append(trueutil)
    newtable=pd.DataFrame(data)
    return newtable

In [100]:
def getstoputil(valrem,util,itera,column1,column2,value):
    val=valrem[valrem['iteration']==itera][column1].get_values()
    mask=val<value
    if mask.any():
        loc=(np.where(mask)[0][0]+1)*20
    else:
        loc=1020
    trueutil=util[(util['iteration']==itera) & (util['respondent']==loc)][column2].get_values()[0]
    return loc,trueutil

In [101]:
stopingutils(items='120')


In [102]:
stopingutils(items='40')
stopingutils(items='300')

In [10]:
trueutil=pd.read_csv('../MABMaxDiff/HB_120items.csv').get_values().mean(0)[2:]

In [12]:
plt.plot(range(1,121),np.sort(trueutil),'k.')
plt.plot(range(111,121),np.sort(trueutil)[-10:],'b.')
plt.plot(range(118,121),np.sort(trueutil)[-3:],'r.')
#plt.title('Utility Score of 120 Items')
plt.ylabel('Utility')
plt.xlabel('Items')
plt.xlim(0,121)
plt.savefig('Utilscore.pdf')
plt.show()

In [18]:
plt.hist(trueutil,bins=20)
plt.show()